# Optimizing the code

### Authors: [Natalí S. M. de Santi](https://natalidesanti.github.io/) and [Christopher Lovell](https://www.christopherlovell.co.uk/)

We can obtain better results doing a **hyperparameter optimization** and we can do this using `optuna`, a framework for automating the optimization process of these hyperparameters.

The main idea is to use a metric, from your validation set, to look for the best set of hyperparameters:

![](https://editor.analyticsvidhya.com/uploads/67819opt.png)

Let's do it!

## 0. Necessary libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import seaborn as sns
from keras import models
from keras import layers
from keras import regularizers
import os
from scipy.stats import gaussian_kde
from numpy.random import seed
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna

## 1. Reading the data

In [3]:
df_train = pd.read_csv('../data/train-halos_and_gals.csv')
df_test = pd.read_csv('../data/test-halos_and_gals.csv')

Using:
* halos: mass, radius, velocity modulus

To predict
* galaxies: stellar masses

\begin{equation}
 \{ M_h, R_h, V_h \} \Rightarrow M_{\star}
\end{equation}

In [4]:
x_train = np.array([df_train['M_h'], df_train['R_h'], df_train['V_h']]).T
x_test = np.array([df_test['M_h'], df_test['R_h'], df_test['V_h']]).T

y_train = np.array([df_train['M_g']]).T
y_test = np.array([df_test['M_g']]).T

## 2. Pre-processing the data

We will do the following transformation
\begin{equation}
 data \Rightarrow \frac{(data - mean)}{std}
\end{equation}

In [5]:
#Taking mean and std
mean_data = np.mean(x_train, axis = 0)
std_data = np.std(x_train, axis = 0)
mean_data, std_data

(array([10.72031404, -6.69246455,  2.14441201]),
 array([0.57831041, 0.19277028, 0.24073891]))

In [6]:
x_train -= mean_data
x_train /= std_data
x_test -= mean_data
x_test /= std_data

In [6]:
def objective(trial):
    
    n_layers = trial.suggest_int('n_layers', 0, 5)
    hiddens = trial.suggest_int('hiddens', 0, 64)
    
    def base_model():
        model = models.Sequential() 
        model.add(layers.Dense(hiddens, activation = 'relu', input_shape = (x_train.shape[1],)))
        for i in range(n_layers):
            model.add(layers.Dense(hiddens, activation = 'relu'))
        model.add(layers.Dense(3))
        
        model.compile(optimizer = 'Adamax', loss = 'mse')
    
        return model
    
    my_model = KerasRegressor(build_fn = base_model)
    
    no_epochs = 20
    batch_size = 20
    validation_split = 0.2
    history = my_model.fit(x_train, y_train, epochs = no_epochs, batch_size = batch_size, 
                 validation_split = validation_split, verbose = False)

    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    validation = 10000.
    
    for i in range(no_epochs):
        if val_loss[i] < train_loss[i]:
            validation = val_loss[i]
            
            my_model.model.save('results/optuna_model.h5')
        
    return validation

In [7]:
study = optuna.create_study(direction = 'minimize')
study.optimize(objective, n_trials = 1)

[I 2023-01-19 10:47:55,409] A new study created in memory with name: no-name-422bfecc-3630-41d6-846a-ad87c29923e7
[I 2023-01-19 10:47:59,256] Trial 0 finished with value: 9.809013366699219 and parameters: {'n_layers': 0, 'hiddens': 11}. Best is trial 0 with value: 9.809013366699219.
